In [128]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import time
import string
from collections import defaultdict, deque
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt

In [129]:
input_dir=r"C:\Users\Lenovo\Desktop\stupidcode\kaggle\jigsaw\error_analyze"
aug1_paths=os.path.join(input_dir,f"aug_1\\data_df_aug1")


In [130]:

data_df=pd.read_csv(aug1_paths)
data_df["label_min"]=data_df.apply(lambda row:row["encode_less"] if row["encode_less"]<row["encode_more"] else row["encode_more"],axis=1)
data_df["label_max"]=data_df.apply(lambda row:row["encode_less"] if row["encode_less"]>row["encode_more"] else row["encode_more"],axis=1)
data_df["label_min_text"]=data_df.apply(lambda row:row["less_toxic"] if row["encode_less"]<row["encode_more"] else row["more_toxic"],axis=1)
data_df["label_max_text"]=data_df.apply(lambda row:row["less_toxic"] if row["encode_less"]>row["encode_more"] else row["more_toxic"],axis=1)
data_df["label_min_value"]=data_df.apply(lambda row:row["less_value"] if row["encode_less"]<row["encode_more"] else row["more_value"],axis=1)
data_df["label_max_value"]=data_df.apply(lambda row:row["less_value"] if row["encode_less"]>row["encode_more"] else row["more_value"],axis=1)


# data_df["pair_True"].replace({True:1,False:-1},inplace=True)

In [131]:
data_df.head(3)

,Unnamed: 0,worker,less_toxic,more_toxic,encode_less,encode_more,label_min,label_max,win_min,win_max,kfold,less_value,more_value,pair_True,label_min_text,label_max_text,label_min_value,label_max_value
0,0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,2405,12151,2405,12151,0.0,1.0,4,0.203842,0.676673,True,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.203842,0.676673
1,1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,7215,653,653,7215,1.0,0.0,0,-0.132469,0.321412,True,Daphne Guinness \n\nTop of the mornin' my fav...,"""And yes, people should recognize that but the...",0.321412,-0.132469
2,2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",2632,7222,2632,7222,0.0,1.0,0,0.194184,0.021682,False,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.194184,0.021682


In [132]:
origin_data_df=data_df[data_df["worker"]!=999]
aug_data_df=data_df[data_df["worker"]==999]

print("origin_cv:",origin_data_df["pair_True"].mean())
print("aug_data_cv:",aug_data_df["pair_True"].mean())



origin_cv: 0.757373455559984
aug_data_cv: 0.9073312363900314


In [71]:
# origin_error_df=origin_data_df[origin_data_df["pair_True"]==False]
# aug_error_df=aug_data_df[aug_data_df["pair_True"]==False]
# #找到所有的false 组合的情况（包括被判断正确的组合）
# origin_error_df=origin_data_df[origin_data_df["pair_True"]==False]
# false_pair=[[ row["label_min"],row["label_max"] ] for index,row in origin_error_df.iterrows() ]
# origin_data_df["is_lost"]=origin_data_df.apply(lambda row:True if [row["label_min"],row["label_max"]] in false_pair 
#                                                and row["pair_True"]==True else False ,axis=1)
# origin_error_df=pd.concat([origin_data_df[origin_data_df["is_lost"]],origin_error_df],axis=0)
# origin_error_df_agg=origin_error_df.groupby(["label_min","label_max"]).agg({"win_min":"sum","win_max":"sum"}).reset_index()
# origin_error_df_agg["encode_less"]=origin_error_df_agg.apply(lambda row:row["label_min"] 
#                                              if row["win_min"]<row["win_max"] else row["label_max"],axis=1)
# origin_error_df_agg["encode_more"]=origin_error_df_agg.apply(lambda row:row["label_min"] 
#                                              if row["win_min"]>row["win_max"] else row["label_max"],axis=1)

# # 92%的错误都来自 毒性相似句子 所有句子对中 52%是毒性相似句子
# confuse_error_df=origin_error_df_agg[(origin_error_df_agg["win_min"]!=0) & (origin_error_df_agg["win_max"]!=0)]
# len(confuse_error_df)/len(origin_error_df_agg)

In [147]:
origin_data_df_agg=origin_data_df.groupby(["label_min","label_max","label_min_text","label_max_text"
                                        ]).agg({"win_min":"sum","win_max":"sum","label_min_value":"mean","label_max_value":"mean","pair_True":"mean"}).reset_index()
origin_data_df_agg["value_gap"]=origin_data_df_agg.apply(lambda row:row["label_max_value"]-row["label_min_value"] 
                                                         if row["win_max"]>row["win_min"] else row["label_min_value"]-row["label_max_value"],axis=1)
origin_data_df_agg["win_gap"]=origin_data_df_agg.apply(lambda row:(row["win_max"]-row["win_min"])/(row["win_max"]+row["win_min"])
                                                         if row["win_max"]>row["win_min"] else (row["win_min"]-row["win_max"])/(row["win_max"]+row["win_min"]),axis=1)

In [148]:
origin_data_df_agg.head()

,label_min,label_max,label_min_text,label_max_text,win_min,win_max,label_min_value,label_max_value,pair_True,value_gap,win_gap
0,0,7085,\n\nThe comment directly above this one are fr...,"""::you wouldn't know Makoto from Street Fighte...",2.0,1.0,-0.075334,-0.183623,0.666667,0.108288,0.333333
1,1,1293,\n\nwhy should people have to read crap posted...,I just read what you wrote; neither of the tw...,3.0,0.0,0.487768,-0.248887,1.000000,0.736656,1.000000
2,1,5156,\n\nwhy should people have to read crap posted...,"""\n\nI don't know which """"personal attacks"""" y...",3.0,0.0,0.487768,-0.509758,1.000000,0.997527,1.000000
3,1,12696,\n\nwhy should people have to read crap posted...,"Yes it is. This is NOT an encyclopedia, it is ...",2.0,1.0,0.487768,0.208761,0.666667,0.279007,0.333333
4,2,13776,\nGo F yourself you cottonheadednittymuggins.,people like you are the reason wikipedia is us...,2.0,1.0,0.813417,-0.380262,0.666667,1.193679,0.333333


In [149]:
#2291 条数据错误 7817条正确
origin_data_error=origin_data_df_agg[origin_data_df_agg["pair_True"]<0.5]
origin_data_true=origin_data_df_agg[origin_data_df_agg["pair_True"]>0.5]
len(origin_data_true)/len(origin_data_df_agg)

0.8396319746735259

In [150]:
# origin_data_error.to_csv(os.path.join(input_dir,f"aug_1\\origin_data_error.csv"))
# origin_data_true.to_csv(os.path.join(input_dir,f"aug_1\\origin_data_true.csv"))

In [206]:
dict(origin_data_true["win_gap"].value_counts())[1/3]

3905

In [275]:
#对正确的部分 划分为正1 正2 根据赢多少的比例来
win_gap_num=dict(origin_data_true["win_gap"].value_counts())
true_point=win_gap_num[1/3]/(win_gap_num[1/3]+win_gap_num[1])
true_val_gap=list(origin_data_true["value_gap"].sort_values())
true_point_value=true_val_gap[int(true_point*len(true_val_gap))]
origin_data_true["degree"]=origin_data_true.apply(lambda row:2 if row["value_gap"]>true_point_value else 1,axis=1)

origin_true_degree2=origin_data_true[origin_data_true["degree"]==2]
origin_true_degree1=origin_data_true[origin_data_true["degree"]==1]

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [271]:
#对错误的部分 也划分为 负1 和负2
false_gap_num=dict(origin_data_error["win_gap"].value_counts())
false_point=false_gap_num[1/3]/(false_gap_num[1/3]+false_gap_num[1])
false_val_gap=list(origin_data_error["value_gap"].sort_values())
false_point_value=false_val_gap[int(false_point*len(false_val_gap))]
origin_data_error["degree"]=origin_data_error.apply(lambda row:1 if row["value_gap"]>false_point_value else 2,axis=1)

origin_false_degree2=origin_data_error[origin_data_error["degree"]==2]
origin_false_degree1=origin_data_error[origin_data_error["degree"]==1]

D:\anaconda\envs\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 绘图